# Classifiez automatiquement des biens de consommation
## Notebook 4 : Collecte de produits
OpenClassrooms - Parcours Data Scientist - Projet 06  

## Présentation du projet

**Contexte**  


* **Place de marché** souhaite lancer une marketplace e-commerce anglophone  
* Les vendeurs proposent des articles à des acheteurs en postant une photo et une description  
* Pour l'instant, l'attribution de la catégorie d'un article est effectuée manuellement par les vendeurs, elle est peu fiable. Le volume des articles est très petit  
* Il devient nécessaire d'**automatiser la tâche d‘attribution de la catégorie**

**Missions**  
* Étude de faisabilité à partir de données texte (Notebook 1)  
* Étude de faisabilité à partir des images - SIFT (Notebook 2)  
* Étude de faisabilité à partir des images - CNN transfer Learning (Notebook 3)  
* **Collecte de produits** à partie d'une API (Ce notebook)

**Collecte de produits**  
* Tester la collecte de produits à base de “champagne” via l’[API edanam](https://rapidapi.com/edamam/api/edamam-food-and-grocery-database)  
* Extraire les 10 premiers produits dans un fichier “.csv”, contenant pour chaque produit les données suivantes : foodId, label, category, foodContentsLabel, image  

# 1 Test de l'API

* L'url fournie permet d'accéder à une page de test de l'API  
* Après enregistrement de l'utilisateur, une clé API est automatiquement générée et peut être récupérée par un simple copier-coller
* Pour tester la collecte de produits à base de champagne :
    * Effacer le contenu des champs pré-remplis
    * Saisir dans le champ `ingr` la chaîne de caractères 'champagne'
* Une importante liste de snippets est proposée : le choix se porte sur 'Python > requests'
* Ce snippet donne le code permettant d'exécuter et de récupérer le résultat de la requête de collecte des produits à base de champagne :

```python
import requests

url = "https://edamam-food-and-grocery-database.p.rapidapi.com/api/food-database/v2/parser"

querystring = {"ingr":"champagne"}

headers = {
	"X-RapidAPI-Key": "fe68bf5815msh52f466cefae925dp1ad0b8jsn280aff8d1abe",
	"X-RapidAPI-Host": "edamam-food-and-grocery-database.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())
```

* La page propose aussi l'affichage du résultat de la requête, ce qui permet d'analyser la structure de la réponse et de repérer les champs à récupérer :

![](json_products.jpg)

* A noter que lorsqu'une information est manquante pour un produit, le champ correspondant n'est pas présent

# 2 Script de collecte

In [51]:
# Import des module
import requests
import time
import pandas as pd

## 2.1 Interrogation de la base de données

In [41]:
# A mettre si besoin dans un fichier à part
API_KEY = "fe68bf5815msh52f466cefae925dp1ad0b8jsn280aff8d1abe"

In [44]:
# Fonction d'interrogation de la base de données 'edanam'
def query_edanam_databse_by_ingredient(api_key, ingredient='Champagne'):
    """
    Renvoit les produits à base d'un ingrédient de la BDD 'edanam'

    Parameters:
        api_key (str)    : Clé API pour la connexion à la BDD edanam
        ingredient (str) : Critère de sélection des produits

    Returns:
        query_result (JSON) : Liste des produits au format JSON
                              (None en cas d'erreur)
    """
    # Endpoint
    url = "https://edamam-food-and-grocery-database.p.rapidapi.com/api/food-database/v2/parser"

    # Requête
    querystring = {"ingr": ingredient}

    # Headers obligatoires
    headers = {
    	"X-RapidAPI-Key": api_key,
    	"X-RapidAPI-Host": "edamam-food-and-grocery-database.p.rapidapi.com"
    }

    # Récupération de la réponse
    response = requests.get(url, headers=headers, params=querystring)

    # Vérification du statut de réponse
    query_result = None
    if response.status_code == 200:
        query_result = response.json()

    return query_result

## 2.2 Traitement de la réponse

* Éxécution de la requête

In [49]:
start = time.perf_counter()

# Récupération des produits
json_products = query_edanam_databse_by_ingredient(API_KEY, ingredient="champagne")

print(f"Durée d'exécution de la requête : {round(time.perf_counter() - start, 2)} secondes.")

Durée d'exécution de la requête : 0.86 secondes.


* Enregistrement des champs 'foodId', 'label', 'category', 'foodContentsLabel' et 'image' dans un DataFrame

In [50]:
# Nombre de produits à récupérer
numbe_of_products = 10

# Dictionnaire pour stocker les résultats
extraction = {}

# Récupération des champs
for num_product in range(numbe_of_products):
    datas = products[num_product]['food']
    for field in ('foodId', 'label', 'category', 'foodContentsLabel', 'image'):
        extraction.setdefault(field, []).append(datas.get(field, ''))

# Conversion du dictionnaire en DataFrame
products = pd.DataFrame(extraction)

* Enregistrement 'edanam_products.csv'

In [54]:
# Enregistrement au format 'csv'
products.to_csv('edanam_products.csv', index=False)

## 2.3 Vérification

In [55]:
# Chargement du fichier dans un DataFrame
df_products = pd.read_csv('edanam_products.csv')

In [56]:
# Affichage du contenu
display(df_products)

,foodId,label,category,foodContentsLabel,image
0,food_a656mk2a5dmqb2adiamu6beihduu,Champagne,Generic foods,NaN,https://www.edamam.com/food-img/a71/a718cf3c52...
1,food_b753ithamdb8psbt0w2k9aquo06c,"Champagne Vinaigrette, Champagne",Packaged foods,OLIVE OIL; BALSAMIC VINEGAR; CHAMPAGNE VINEGAR...,NaN
2,food_b3dyababjo54xobm6r8jzbghjgqe,"Champagne Vinaigrette, Champagne",Packaged foods,INGREDIENTS: WATER; CANOLA OIL; CHAMPAGNE VINE...,https://www.edamam.com/food-img/d88/d88b64d973...
3,food_a9e0ghsamvoc45bwa2ybsa3gken9,"Champagne Vinaigrette, Champagne",Packaged foods,CANOLA AND SOYBEAN OIL; WHITE WINE (CONTAINS S...,NaN
4,food_an4jjueaucpus2a3u1ni8auhe7q9,"Champagne Vinaigrette, Champagne",Packaged foods,WATER; CANOLA AND SOYBEAN OIL; WHITE WINE (CON...,NaN
5,food_bmu5dmkazwuvpaa5prh1daa8jxs0,"Champagne Dressing, Champagne",Packaged foods,SOYBEAN OIL; WHITE WINE (PRESERVED WITH SULFIT...,https://www.edamam.com/food-img/ab2/ab2459fc2a...
6,food_alpl44taoyv11ra0lic1qa8xculi,Champagne Buttercream,Generic meals,sugar; butter; shortening; vanilla; champagne;...,NaN
7,food_byap67hab6evc3a0f9w1oag3s0qf,Champagne Sorbet,Generic meals,Sugar; Lemon juice; brandy; Champagne; Peach,NaN
8,food_am5egz6aq3fpjlaf8xpkdbc2asis,Champagne Truffles,Generic meals,butter; cocoa; sweetened condensed milk; vanil...,NaN
9,food_bcz8rhiajk1fuva0vkfmeakbouc0,Champagne Vinaigrette,Generic meals,champagne vinegar; olive oil; Dijon mustard; s...,NaN
